## Постановка задачи
Загрузим данные и подготовим все данные для анализа: проведем нормализацию и преобразование категорий. Оптимизируем потребление памяти.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Применим наивный Байес для классификации скоринга. Будем использовать все возможные столбцы.

Проверим качество предсказания через каппа-метрику и матрицу неточностей.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


###  Категоризация данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 129 entries, Id to Product_Info_2_2
dtypes: float64(18), int64(110), object(1)
memory usage: 58.4+ MB
None


### Оптимизация потребления памяти

In [8]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print ("Потребление памяти меньше на", round(start_mem - end_mem, 2), "Мб (минус)", round(100*(start_mem - end_mem) / start_mem, 1), "%)")
    return df

In [9]:
data = reduce_mem_usage(data)
print (data.info())

Потребление памяти меньше на 49.89 Мб (минус) 85.4 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 129 entries, Id to Product_Info_2_2
dtypes: category(1), float16(18), int16(1), int32(1), int8(108)
memory usage: 8.6 MB
None


### Предобработка: категоризация, единичные векторы
| Product |
| - |
| A |
| B |
| C |
| A |

Переходит в

| ProductA | ProductB | ProductC |
| -- | -- | -- |
| 1 | 0 | 0 |
| 0 | 1 | 0 |
| 0 | 0 | 1 |
| 1 | 0 | 0 |

Можно использовать sklearn.preprocessing.OneHotEncoder, но для этого потребуется дополнительно преобразовать фрейм данных (набор единичных векторов для каждого кортежа данных).

Также не будем использовать кодирование категорий (A->1, B->2, C->3, D->4, E->5), потому что это переводит номинативную случайную величину в ранговую/числовую, и является существенным допущением относительно исходных данных.

In [10]:
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)

### Заполним отсутствующие значения
-1 увеличивает "расстояние" при расчете ближайших соседей

In [12]:
data.fillna(value=-1, inplace=True)

### Столбцы для модели

In [13]:
columns_groups = ["Insurance_History", "InsuredInfo", "Medical_Keyword",
                 "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keywor

### Предобработка данных
Дополнительно проведем z-нормализацию данных через предварительную обработку (preprocessing). Нормализуем весь исходный набор данных.

In [14]:
scaler = preprocessing.StandardScaler()
scaler.fit(pd.DataFrame(data, columns=columns))

StandardScaler(copy=True, with_mean=True, with_std=True)

### Разделение данных

In [15]:
data_train, data_test = train_test_split(data, test_size=0.2)
print (data_train.head())

          Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
51154  68123               1              26        0.076904               2   
37067  49216               1              29        0.487061               2   
11378  15091               1              26        0.076904               2   
5068    6749               1              37        0.076904               2   
34965  46443               1              26        0.128174               2   

       Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
51154               3               1  0.313477  0.781738  0.592285  ...   
37067               3               1  0.193970  0.763672  0.435059  ...   
11378               3               1  0.761230  0.545410  0.194580  ...   
5068                3               1  0.029846  0.618164  0.236450  ...   
34965               3               1  0.522461  0.763672  0.299072  ...   

       Medical_Keyword_46  Medical_Keyword_47  Medical_Keyword

### Расчет модели наивного Байеса
\begin{equation}
P(A\mid B) = \frac{P(B\mid A)\ P(A)}{P(B)}
\end{equation}
Для каждого параметра вычисляется его вероятность принять определенное значение - P(B). Для каждого класса вычисляется его вероятность (по факту, доля) - P(A). Затем вычисляется вероятность для каждого параметра принять определенное значение при определенном классе - P(B\A).

По всем вычисленным значениям находится вероятность при известных параметрах принять какое-либо значение класса.

In [18]:
y = data_train["Response"]
x = scaler.transform(pd.DataFrame(data_train, columns=columns))

In [19]:
bayes = GaussianNB()
bayes.fit(x, y)

GaussianNB(priors=None, var_smoothing=1e-09)

### Предсказание данных

In [20]:
data_test = pd.DataFrame(data_test)
x_test = scaler.transform(pd.DataFrame(data_test, columns=columns))
data_test["target"] = bayes.predict(x_test)
print (data_test.head())

          Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
12178  16185               1              26        0.154907               2   
29527  39272               1              26        1.000000               2   
8574   11422               1              15        0.230713               2   
27890  37150               1              26        0.230713               2   
8851   11793               2              26        1.000000               2   

       Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
12178               3               1  0.626953  0.799805  0.386963  ...   
29527               3               1  0.253662  0.836426  0.351562  ...   
8574                3               1  0.283691  0.745605  0.320068  ...   
27890               3               1  0.223877  0.745605  0.267822  ...   
8851                1               1  0.522461  0.708984  0.414307  ...   

       Medical_Keyword_47  Medical_Keyword_48  Response  Produ

### Оценка модели

In [21]:
print ("Байес:", cohen_kappa_score(data_test["target"],
            data_test["Response"], weights="quadratic"))

Байес: 0.1993841553537673


### Матрица неточностей

In [22]:
print (confusion_matrix(data_test["target"], data_test["Response"]))

[[ 289  212   10   12   93  178   79  112]
 [ 110  170    3    3   52   79   32   35]
 [ 102  115   17    8   58   85   19   35]
 [ 317  338  115  233  470 1021  700 2149]
 [  44   75    2    1   26   34    8   10]
 [  13   19    0    1    4   17    2    3]
 [ 340  376   32   43  396  739  678  812]
 [  44   20    5    8   32   76   94  672]]
